In [ ]:
# return the Canny edge detection of image
def getEdges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    return edged

In [ ]:
# return the Canny edge detection of Table by increase the height parameter
def getEdgesTable(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 1)
    edged = cv2.Canny(blurred, 30, 100)
    return edged

In [ ]:
# return the biggest square Contour of image
def getContours(edged):
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    docCnt = None
    boarderd = []
    if len(cnts) > 0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            points = len(approx)
            if (points == 4):
                docCnt = approx
                break
    return docCnt

In [ ]:
# remove the border of Image here it used for TAbles
def CropImg(img):
    cropped_image = img[int(img.shape[0] * .01):int(img.shape[0] * .98), int(img.shape[1] * .02):int(img.shape[1] * .98)]
    return cropped_image

In [ ]:
#extract the Table from the image and modify point of view
def exetractTable(image,docCnt):
    Table = four_point_transform(image, docCnt.reshape(4, 2))
    return Table

In [ ]:
# frinction to remove the internal details of the image to make the paper itself the bigest contour 
def RemoveText(image):
    kernel = np.ones((8,8),np.uint8)
    img = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations= 3)
    return img

In [ ]:
# get little wider table to show the outlines vertical lines
def widerTable(docCnt, imageShape):
    heightincrese = int(imageShape[0]*.015)
    widthincrese = int(imageShape[1]*.01)
    halfHor = int(imageShape[1]/2)
    halfVer = int(imageShape[0]/2)
    
    widerdoc = np.copy(docCnt)
    for c in docCnt:
        c[0][0] += (c[0][0] > halfHor) * widthincrese
        c[0][0] -= (c[0][0] < halfHor) * widthincrese
        
        c[0][1] += (c[0][1] > halfVer) * heightincrese
        c[0][1] -= (c[0][1] < halfVer) * heightincrese
        
    return docCnt

In [ ]:
# remove all hars that neasr to each other by 5px
def removeReplication(boxes, ratio):
    validboxes = []
    Widths =[]
    AVGwidth = 0
    validboxes.append(boxes[0])
    for i in range(1, len(boxes)):
        if(abs(boxes[i][0] - boxes[i-1][0]) > ratio ):
            validboxes.append(boxes[i])
            Widths.append(abs(boxes[i][0] - boxes[i][1]))
            AVGwidth += abs(boxes[i][0] - boxes[i][1])
    Widths.sort()
    return validboxes , Widths[len(Widths)//2] , AVGwidth // len(validboxes)